# Metabolite fluxes and biomass under pFBA and FVA across infection statuses, *E. coli* in monoculture

Set up the necessary environment.

In [1]:
import cobra
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
#set the paths for loading the model and exporting the data
model_path = "../models/iJO1366_plasmid_phage.xml"
export_path = "../fba-data/figure-1/cobra/"
ef1 = cobra.io.read_sbml_model(model_path)

In [18]:
ef1.reactions.plasmid_F.reaction

'1.04306757175046 ala__L_c + 0.373193694583004 arg__L_c + 0.250705367998874 asn__L_c + 0.22892622664389 asp__L_c + 31.2729006765209 atp_c + 0.0614721065047087 cys__L_c + 0.000306008296322521 datp_c + 0.000308808257224839 dctp_c + 0.000308808257224839 dgtp_c + 0.000306008296322521 dttp_c + 0.238187369043553 gln__L_c + 0.166034912599756 glu__L_c + 0.547625048163282 gly_c + 31.2698405935577 h2o_c + 0.108905695544869 his__L_c + 0.301602627712357 ile__L_c + 0.950236446570275 leu__L_c + 0.317027151701196 lys__L_c + 0.423481070021571 met__L_c + 0.476659987338735 phe__L_c + 0.221033209317653 pro__L_c + 0.52578284299389 ser__L_c + 0.386358039411312 thr__L_c + 0.219850756328091 trp__L_c + 0.143534017921975 tyr__L_c + 0.833776083872902 val__L_c --> 31.2698405935577 adp_c + 31.2698405935577 h_c + 31.2698405935577 pi_c + 0.0122962793156625 ppi_c'

## Get the biomass reactions to compare components across host and parasites

In [ ]:
def parse_to_dataframe(input_string):
    # Split the input string at " --> " to separate the relevant and ignored parts
    relevant_part = input_string.split(" --> ")[0]
    
    # Split the relevant part by '+' to separate each term
    terms = relevant_part.split(" + ")
    
    # Initialize lists to store coefficients and chemical terms
    coefficients = []
    chemicals = []
    
    # Process each term
    for term in terms:
        # Split the term by whitespace to separate the coefficient and the chemical term
        coefficient, chemical = term.split(" ", 1)
        coefficients.append(float(coefficient))
        chemicals.append(chemical)
    
    # Create a DataFrame with two columns
    df = pd.DataFrame({
        'Coefficient': coefficients,
        'Chemical': chemicals
    })
    
    return df

In [ ]:
input_string = ef1.reactions.BIOMASS_Ec_iJO1366_core_53p95M.reaction
df_host = parse_to_dataframe(input_string)
df_host["reaction"] = "host"

In [ ]:
input_string = ef1.reactions.phage_M13.reaction
df_virus = parse_to_dataframe(input_string)
df_virus["reaction"] = "virus"

In [ ]:
input_string = ef1.reactions.plasmid_F.reaction
df_plasmid = parse_to_dataframe(input_string)
df_plasmid["reaction"] = "plasmid"

In [ ]:
df = pd.concat([df_host, df_virus, df_plasmid])

## Find the maximum bacterial biomass that can be generated when parasite biomass is optimized

In [3]:
ef1 = cobra.io.read_sbml_model(model_path)
medium = ef1.medium
medium["EX_glc__D_e"] = 0.0
medium["EX_lcts_e"] = 10.0
medium["EX_met__L_e"] = 10.0
ef1.medium = medium
ef1.genes.b3939.knock_out()
ef1.objective = ef1.reactions.phage_M13 #set the objective to viral biomass
solution = ef1.optimize()
maximized_f1_gluc = solution.fluxes.phage_M13

ef1 = cobra.io.read_sbml_model(model_path)
medium = ef1.medium
medium["EX_glc__D_e"] = 0.0
medium["EX_lcts_e"] = 10.0
medium["EX_met__L_e"] = 10.0
ef1.medium = medium
ef1.genes.b3939.knock_out()
ef1.objective = ef1.reactions.plasmid_F #set the objective to plasmid biomass
solution = ef1.optimize()
maximized_plasmid_gluc = solution.fluxes.plasmid_F

## Get fluxes using pFBA in either a host or parasite optimized state

In [4]:
ef1 = cobra.io.read_sbml_model(model_path)
medium = ef1.medium
medium["EX_glc__D_e"] = 0.0
medium["EX_lcts_e"] = 10.0
medium["EX_met__L_e"] = 10.0
ef1.medium = medium
ef1.genes.b3939.knock_out()
ef1.objective = ef1.reactions.get_by_id("BIOMASS_Ec_iJO1366_core_53p95M")
solution = ef1.optimize()
host_fluxes = pd.DataFrame(solution.fluxes)
host_fluxes["parasite"] = "bacteria"

In [5]:
ef1 = cobra.io.read_sbml_model(model_path)
medium = ef1.medium
medium["EX_glc__D_e"] = 0.0
medium["EX_lcts_e"] = 10.0
medium["EX_met__L_e"] = 10.0
ef1.medium = medium
ef1.genes.b3939.knock_out()
ef1.objective = ef1.reactions.plasmid_F #set the objective to plasmid biomass
solution = ef1.optimize()
plasmid_fluxes = pd.DataFrame(solution.fluxes)
plasmid_fluxes["parasite"] = "plasmid"

In [6]:
pfba_optimized = pd.concat([host_fluxes, plasmid_fluxes])

In [7]:
range_plasmid_fluxes = pd.DataFrame()
for i in np.arange(0, maximized_plasmid_gluc, 0.1):
    ef1 = cobra.io.read_sbml_model(model_path)
    medium = ef1.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_lcts_e"] = 10.0
    medium["EX_met__L_e"] = 10.0
    ef1.medium = medium
    ef1.genes.b3939.knock_out()
    ef1.objective = ef1.reactions.phage_M13 #set the objective to viral biomass given a plasmid lower bound
    ef1.reactions.plasmid_F.lower_bound = i
    solution = ef1.optimize()
    plasmidphage_fluxes = pd.DataFrame(solution.fluxes)
    plasmidphage_fluxes["parasite"] = "virus with plasmid"
    plasmidphage_fluxes["plasmid_bound"] = i
    range_plasmid_fluxes = pd.concat([range_plasmid_fluxes, plasmidphage_fluxes])

## Get bacterial fluxes using FVA in a host or parasite optimized state

In [8]:
#host only
host_fva = pd.DataFrame()
ef1 = cobra.io.read_sbml_model(model_path)
ef1.genes.b3939.knock_out()
medium = ef1.medium
medium["EX_glc__D_e"] = 0.0
medium["EX_lcts_e"] = 10.0
medium["EX_met__L_e"] = 10.0
ef1.medium = medium
ef1.objective = ef1.reactions.get_by_id("BIOMASS_Ec_iJO1366_core_53p95M")
solution = cobra.flux_analysis.flux_variability_analysis(ef1, ef1.reactions, fraction_of_optimum=0.95)
host_fva = pd.concat([host_fva, solution])

host_fva["optimized"] = "host"

In [9]:
#plasmid only
plasmid_fva = pd.DataFrame()
ef1 = cobra.io.read_sbml_model(model_path)
ef1.genes.b3939.knock_out()
medium = ef1.medium
medium["EX_glc__D_e"] = 0.0
medium["EX_lcts_e"] = 10.0
medium["EX_met__L_e"] = 10.0
ef1.medium = medium
ef1.objective = ef1.reactions.plasmid_F
solution = cobra.flux_analysis.flux_variability_analysis(ef1, ef1.reactions, fraction_of_optimum=0.95)
plasmid_fva = pd.concat([plasmid_fva, solution])

plasmid_fva["optimized"] = "plasmid"

In [10]:
fva_fluxes_optimized = pd.concat([plasmid_fva,host_fva])

In [11]:
fvarange_plasmid_fluxes = pd.DataFrame()
for i in np.arange(0, maximized_plasmid_gluc, 0.1):
    plasmidphage_fva = pd.DataFrame()
    ef1 = cobra.io.read_sbml_model(model_path)
    ef1.genes.b3939.knock_out()
    medium = ef1.medium
    medium["EX_glc__D_e"] = 0.0
    medium["EX_lcts_e"] = 10.0
    medium["EX_met__L_e"] = 10.0
    ef1.medium = medium
    ef1.objective = ef1.reactions.phage_M13
    ef1.reactions.plasmid_F.lower_bound = i
    solution = cobra.flux_analysis.flux_variability_analysis(ef1, ef1.reactions, fraction_of_optimum=0.95)
    plasmidphage_fva = pd.concat([plasmidphage_fva, solution])
    plasmidphage_fva["optimized"] = "virus with plasmid"
    plasmidphage_fva["plasmid_bound"] = i
    fvarange_plasmid_fluxes = pd.concat([fvarange_plasmid_fluxes, plasmidphage_fva])

## Subprocesses for categorization

In [ ]:
subprocesses = pd.DataFrame()
for x in ef1.groups:
    temp = pd.DataFrame()
    temp["reaction"] = [x.members[i].id for i in range(0, len(x.members))]
    temp["group_name"] = x.name
    temp["id"] = x.id
    subprocesses = subprocesses.append(temp)

# Finally, save and export all data files for analysis in R.

In [12]:
pfba_optimized.to_csv(export_path + 'pfba_fluxes_optimized.csv')
fva_fluxes_optimized.to_csv(export_path + 'fva_fluxes_optimized.csv')
#subprocesses.to_csv(export_path + 'subprocesses.csv') # cellular subprocesses
df.to_csv(export_path + 'core_reactions.csv') #core biomass metabolites
range_plasmid_fluxes.to_csv(export_path + 'pfba_fluxes_virusplasmid_plasmidrange_virusoptimized.csv') # change plasmid production range while maximizing phage
fvarange_plasmid_fluxes.to_csv(export_path + 'fva_fluxes_virusplasmid_plasmidrange_virusoptimized.csv') # change plasmid production range while maximizing phage

NameError: name 'df' is not defined